In [2]:
# jupyter 상에서 SVG 이미지를 표시하기 위한 패키지(jupyter 기본 내장 패키지)
from IPython.display import SVG

# TAG로부터 원하는 내용을 추출하는 클래스 -> SVG 이미지의 핸들링을 위함
from bs4 import BeautifulSoup

# 원격지의 svg 파일의 내용을 읽어오기 위함.
import requests

from pandas import pivot_table, read_csv, read_excel
import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sb
import folium
import folium
from os import path, mkdir

In [3]:
plt.rcParams["font.family"] = 'Malgun Gothic'
#plt.rcParams["font.family"] = 'AppleGothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (20, 5)
plt.rcParams["axes.unicode_minus"] = False

In [4]:
ha = read_csv('C:/Users/User/Desktop/세종특별자치시_CCTV.csv', encoding='cp949')
ha


,관리기관명,소재지지번주소,설치목적구분,카메라대수,카메라화소수,촬영방면정보,보관일수,관리기관전화번호,위도,경도,데이터기준일자
0,세종특별자치시,세종특별자치시 세종동 981,생활방범,2,200,세종리 은행나무역사공원 입구남측,30,044-300-2454,36.503216,127.290461,2023-07-04
1,세종특별자치시,세종특별자치시 세종동 855-1,생활방범,2,200,세종리 은행나무역사공원 동측 금강사이도로,30,044-300-2454,36.501924,127.296971,2023-07-04
2,세종특별자치시,세종특별자치시 세종동 1201,생활방범,3,200,세종호수공원 B코스,30,044-300-2454,36.498162,127.277682,2023-07-04
3,세종특별자치시,세종특별자치시 세종동 1201,생활방범,3,200,세종호수공원 A코스,30,044-300-2454,36.494496,127.271841,2023-07-04
4,세종특별자치시,세종특별자치시 세종동 1201,생활방범,3,200,중앙공원 도시 전망대 앞,30,044-300-2454,36.493051,127.271102,2023-07-04
...,...,...,...,...,...,...,...,...,...,...,...
1313,세종특별자치시,세종특별자치시 조치원읍 봉산리 296-1,차량방범,3,200,송곡리 봉산리 양방향,30,044-300-2454,36.620523,127.269250,2023-07-04
1314,세종특별자치시,충청북도 청주시 흥덕구 오송읍 서평리 54-6,차량방범,3,200,조천교 조치원,30,044-300-2454,36.603655,127.306564,2023-07-04
1315,세종특별자치시,세종특별자치시 조치원읍 신안리 483-30,차량방범,3,200,세종로 전의,30,044-300-2454,36.625404,127.289933,2023-07-04
1316,세종특별자치시,충청북도 청주시 흥덕구 오송읍 봉산리 608-1,차량방범,4,200,조천교 청주,30,044-300-2454,36.603936,127.306516,2023-07-04


In [12]:
ha['카메라대수'].value_counts()

카메라대수
1    455
3    431
2    333
4     87
5     11
6      1
Name: count, dtype: int64

In [5]:
ilist = list(ha.dtypes.index)
ilist

['관리기관명',
 '소재지지번주소',
 '설치목적구분',
 '카메라대수',
 '카메라화소수',
 '촬영방면정보',
 '보관일수',
 '관리기관전화번호',
 '위도',
 '경도',
 '데이터기준일자']

In [8]:
pdf = ha.filter(['소재지지번주소','경도','위도'])
pdf

,소재지지번주소,경도,위도
0,세종특별자치시 세종동 981,127.290461,36.503216
1,세종특별자치시 세종동 855-1,127.296971,36.501924
2,세종특별자치시 세종동 1201,127.277682,36.498162
3,세종특별자치시 세종동 1201,127.271841,36.494496
4,세종특별자치시 세종동 1201,127.271102,36.493051
...,...,...,...
1313,세종특별자치시 조치원읍 봉산리 296-1,127.269250,36.620523
1314,충청북도 청주시 흥덕구 오송읍 서평리 54-6,127.306564,36.603655
1315,세종특별자치시 조치원읍 신안리 483-30,127.289933,36.625404
1316,충청북도 청주시 흥덕구 오송읍 봉산리 608-1,127.306516,36.603936


In [9]:
map_osm = folium.Map(location=[37.566651, 126.978428], zoom_start=12)

html = "<font color='green' style='white-space: nowrap'><b>%s</b></font>"

# 데이터프레임의 행 수만큼 반복하면서 마커 생성
for i in pdf.index:
    # 행 우선 접근 방식으로 값 추출하기
    name = pdf.loc[i, '소재지지번주소']
    lat = pdf.loc[i, '위도']
    lng = pdf.loc[i, '경도']
    
    # 추출한 정보를 지도에 표시
    popup_html = folium.Popup(html % name, parse_html=False)
    marker = folium.Marker([lat,lng], popup=popup_html)
    marker.add_to(map_osm)
map_osm
